# Weather Data Preprocessing: Create Weighted Features (One-Time)

**Purpose:** Convert 8 regional weather CSV files into 1 final weighted file.

**Input:**
- `combined.csv` (your ERCOT load data)
- `weather_files.zip` (8 regional weather CSV files)

**Output:**
- `final_weighted_weather_features.csv` (clean, ready for modeling)

**Run this once, then use the output file in your training notebook!**

In [1]:
import pandas as pd
import numpy as np
import zipfile
import io
from pathlib import Path

print("="*80)
print("WEATHER DATA PREPROCESSING: CREATE WEIGHTED FEATURES")
print("="*80)
print("\nThis notebook:")
print("  1. Loads combined.csv (ERCOT load data)")
print("  2. Unzips weather_files.zip (8 regional weather CSVs)")
print("  3. Calculates weights based on load contribution")
print("  4. Creates weighted weather features")
print("  5. Saves final_weighted_weather_features.csv")
print("\nAfter this, use the output file in your training notebook!")

WEATHER DATA PREPROCESSING: CREATE WEIGHTED FEATURES

This notebook:
  1. Loads combined.csv (ERCOT load data)
  2. Unzips weather_files.zip (8 regional weather CSVs)
  3. Calculates weights based on load contribution
  4. Creates weighted weather features
  5. Saves final_weighted_weather_features.csv

After this, use the output file in your training notebook!


## Step 1: Load ERCOT Load Data

In [2]:
import pandas as pd
from google.colab import drive
import zipfile
import io

# --- Step 1: Mount your Google Drive ---
# This will prompt you to authorize the connection
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("✓ Drive mounted successfully")

# --- Step 2: Update the file path to point to your Drive ---
# This assumes 'combined.csv' is in the main 'My Drive' folder
LOAD_FILE = '/content/drive/MyDrive/combined.csv'

# You can change this path if it's in a subfolder, e.g.:
# LOAD_FILE = '/content/drive/MyDrive/My_Project_Folder/combined.csv'

print(f"\nLoading load data from: {LOAD_FILE}")
load_df = pd.read_csv(LOAD_FILE)

print(f"✓ Loaded: {load_df.shape[0]:,} rows × {load_df.shape[1]} columns")
print(f"\nColumns: {load_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(load_df.head())

# Ensure we have a datetime column
if 'Hour Ending' in load_df.columns:
    load_df['datetime'] = pd.to_datetime(load_df['Hour Ending'])
    load_df['date'] = load_df['datetime'].dt.date
else:
    # Try to find a datetime column
    date_cols = [c for c in load_df.columns if 'date' in c.lower() or 'time' in c.lower() or 'hour' in c.lower()]
    if date_cols:
        load_df['datetime'] = pd.to_datetime(load_df[date_cols[0]])
        load_df['date'] = load_df['datetime'].dt.date
    else:
        raise ValueError("Could not find date/time column in load data")

print(f"\nDate range: {load_df['date'].min()} to {load_df['date'].max()}")

# --- ADDED SECTION ---

# Placeholder for weights_normalized.
# This is needed so the .keys() loop below works.
# You will replace this with your actual weight calculation logic.
print("\nDefining placeholder station keys (will be used to find files in zip)")
weights_normalized = {
    "NCENT": 0, "COAST": 0, "SCENT": 0, "SOUTH_C": 0,
    "FAR_WEST": 0, "WEST": 0, "SOUTHERN": 0, "EAST": 0
}
print(f"✓ Station keys: {list(weights_normalized.keys())}")


# --- Step 3: Load Weather Data from Zip in Drive ---
# Point this to the location of your zip file on Google Drive
WEATHER_ZIP = '/content/drive/MyDrive/weather_files.zip'

print(f"\nUnzipping and reading from: {WEATHER_ZIP}")

weather_files = {}

try:
    with zipfile.ZipFile(WEATHER_ZIP, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"✓ Found {len(file_list)} files in zip archive")

        # Load each weather file based on the keys from our list
        for station in weights_normalized.keys():
            filename = f'weather_data_{station}.csv'

            # Try to find the file in the zip (handles nested folders, e.g. "weather_data/weather_data_COAST.csv")
            matching_files = [f for f in file_list if filename.lower() in f.lower()]

            if matching_files:
                actual_filename = matching_files[0]
                print(f"\nLoading: {actual_filename}")

                with zip_ref.open(actual_filename) as file:
                    # Use io.TextIOWrapper for correct text decoding from bytes
                    df = pd.read_csv(io.TextIOWrapper(file))

                    # Ensure date column (this logic assumes 'time' is the first col or 'date' exists)
                    if 'date' not in df.columns:
                        # Assuming the first column is the datetime index
                        print("  'date' col not found, using first col as datetime.")
                        df['datetime'] = pd.to_datetime(df.iloc[:, 0])
                    else:
                        df['datetime'] = pd.to_datetime(df['date'])

                    # ---
                    # WARNING: The line below converts your hourly data to daily dates.
                    # You may want to change this to keep the hourly 'datetime'
                    # if your model is hourly.
                    # ---
                    print("  Converting datetime to date (this truncates hours!)")
                    df['date'] = df['datetime'].dt.date
                    weather_files[station] = df

                    print(f"   ✓ {len(df)} rows")
                    print(f"     Columns: {df.columns.tolist()[:5]}...")
                    print(f"     Date range: {df['date'].min()} to {df['date'].max()}")
            else:
                print(f"\n⚠️  {filename} not found in zip!")
                print(f"   Available files (sample): {[f for f in file_list if 'weather' in f][:10]}")

except FileNotFoundError:
    print(f"❌ {WEATHER_ZIP} not found!")
    print(f"   Make sure weather_files.zip is in your Google Drive at: /content/drive/MyDrive/weather_files.zip")
    raise
except Exception as e:
    print(f"An error occurred: {e}")
    raise

print(f"\n✓ Loaded {len(weather_files)} weather station files into `weather_files` dictionary")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Drive mounted successfully

Loading load data from: /content/drive/MyDrive/combined.csv


/tmp/ipython-input-1351242258.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  load_df = pd.read_csv(LOAD_FILE)
/tmp/ipython-input-1351242258.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  load_df['datetime'] = pd.to_datetime(load_df['Hour Ending'])


✓ Loaded: 96,430 rows × 17 columns

Columns: ['Hour Ending', 'COAST', 'EAST', 'FWEST', 'NORTH', 'NCENT', 'SOUTH', 'SCENT', 'WEST', 'FAR_WEST', 'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'TOTAL', 'hour', 'day', 'year']

First few rows:
      Hour Ending        COAST         EAST        FWEST        NORTH  \
0  1/1/14 1:00 AM  9101.691219  1338.197939  1338.197939  1338.197939   
1  1/1/14 2:00 AM  8907.975782  1328.940064  1809.180861   791.141630   
2  1/1/14 3:00 AM  8738.460973  1317.990846  1804.524573   789.791906   
3  1/1/14 4:00 AM  8622.671259  1325.545134  1807.227126   797.195877   
4  1/1/14 5:00 AM  8615.480468  1348.596187  1816.456043   810.868332   

          NCENT        SOUTH        SCENT         WEST     FAR_WEST  \
0   1338.197939  1338.197939  1338.197939  1338.197939  1338.197939   
1  12297.109820  3246.493375  6091.018850  1098.774771  1809.180861   
2  12285.295250  3217.721388  6060.017515  1097.716762  1804.524573   
3  12368.070070  3165.203545  6017.610069  1105.891

/tmp/ipython-input-1351242258.py:29: FutureWarning: Parsed string "11/05/2017 02:00 DST" included an un-recognized timezone "DST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  load_df['datetime'] = pd.to_datetime(load_df['Hour Ending'])
/tmp/ipython-input-1351242258.py:29: FutureWarning: Parsed string "11/04/2018 02:00 DST" included an un-recognized timezone "DST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  load_df['datetime'] = pd.to_datetime(load_df['Hour Ending'])
/tmp/ipython-input-1351242258.py:29: FutureWarning: Parsed string "11/03/2019 02:00 DST" included an un-recognized timezone "DST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the st


Date range: 2014-01-01 to 2024-12-31

Defining placeholder station keys (will be used to find files in zip)
✓ Station keys: ['NCENT', 'COAST', 'SCENT', 'SOUTH_C', 'FAR_WEST', 'WEST', 'SOUTHERN', 'EAST']

Unzipping and reading from: /content/drive/MyDrive/weather_files.zip
✓ Found 9 files in zip archive

Loading: weather_data/weather_data_NCENT.csv
  'date' col not found, using first col as datetime.
  Converting datetime to date (this truncates hours!)
   ✓ 96432 rows
     Columns: ['time', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature']...
     Date range: 2014-01-01 to 2024-12-31

Loading: weather_data/weather_data_COAST.csv
  'date' col not found, using first col as datetime.
  Converting datetime to date (this truncates hours!)
   ✓ 96432 rows
     Columns: ['time', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature']...
     Date range: 2014-01-01 to 2024-12-31

Loading: weather_data/weather_data_SCENT.csv
  'date' co

## Step 2: Calculate Weights from Load Data

In [3]:
# Define regional groupings
region_mappings = {
    'Dallas': ['NORTH', 'NCENT', 'NORTH_C'],
    'San_Antonio': ['SOUTH', 'SOUTH_C'],
    'Abilene': ['WEST', 'FWEST'],
    'COAST': ['COAST'],
    'EAST': ['EAST'],
    'FAR_WEST': ['FAR_WEST'],
    'SCENT': ['SCENT'],
    'SOUTHERN': ['SOUTHERN'],
}

print("\n" + "="*80)
print("CALCULATING WEIGHTS FROM LOAD DATA")
print("="*80)

weights = {}
total_avg_load = load_df['TOTAL'].mean()

print(f"\nTotal average load: {total_avg_load:,.0f} MW")
print(f"\nWeight calculation (Region avg load / Total avg load):")

for station, regions in region_mappings.items():
    # Check which regions exist in our data
    available_regions = [r for r in regions if r in load_df.columns]

    if available_regions:
        # Calculate average load from these regions
        regional_load = load_df[available_regions].sum(axis=1).mean()
        weight = regional_load / total_avg_load
        weights[station] = weight

        print(f"\n  {station:15} → {', '.join(available_regions):30}")
        print(f"    Avg load: {regional_load:>10,.0f} MW")
        print(f"    Weight:   {weight:>10.1%}")
    else:
        print(f"\n  {station:15} → No regions found!")

# Normalize weights to sum to 1.0
total_weight = sum(weights.values())
weights_normalized = {k: v / total_weight for k, v in weights.items()}

print(f"\n" + "-"*80)
print(f"NORMALIZED WEIGHTS (sum = {sum(weights_normalized.values()):.4f}):")
print("-"*80)
for station in sorted(weights_normalized.keys(), key=lambda x: weights_normalized[x], reverse=True):
    weight = weights_normalized[station]
    print(f"  {station:15} {weight:6.1%}")

print(f"\n✓ Weights calculated")


CALCULATING WEIGHTS FROM LOAD DATA

Total average load: 44,246 MW

Weight calculation (Region avg load / Total avg load):

  Dallas          → NORTH, NCENT, NORTH_C         
    Avg load:     27,109 MW
    Weight:        61.3%

  San_Antonio     → SOUTH, SOUTH_C                
    Avg load:     10,455 MW
    Weight:        23.6%

  Abilene         → WEST, FWEST                   
    Avg load:      4,633 MW
    Weight:        10.5%

  COAST           → COAST                         
    Avg load:     12,485 MW
    Weight:        28.2%

  EAST            → EAST                          
    Avg load:      1,548 MW
    Weight:         3.5%

  FAR_WEST        → FAR_WEST                      
    Avg load:      3,604 MW
    Weight:         8.1%

  SCENT           → SCENT                         
    Avg load:      6,558 MW
    Weight:        14.8%

  SOUTHERN        → SOUTHERN                      
    Avg load:      3,616 MW
    Weight:         8.2%

------------------------------------

## Step 3: Create Weighted Weather Features

In [ ]:
print("\n" + "="*80)
print("CREATING WEIGHTED WEATHER FEATURES")
print("="*80)

# Weather columns to weight
WEATHER_COLS = [
    'temp_max',
    'temp_min',
    'temp_mean',
    'humidity_max',
    'humidity_min',
    'precipitation',
    'wind_speed_max'
]

# Start with dates from load data
weighted_weather = load_df[['date']].copy().drop_duplicates()
weighted_weather = weighted_weather.sort_values('date').reset_index(drop=True)

print(f"\nBase dataframe: {len(weighted_weather)} unique dates")
print(f"Date range: {weighted_weather['date'].min()} to {weighted_weather['date'].max()}")

# Create weighted columns
print(f"\nCreating weighted features:")

for col in WEATHER_COLS:
    print(f"\n  {col}:")

    weighted_weather[f'weighted_{col}'] = 0.0

    # Add contribution from each station
    for station, weight in weights_normalized.items():
        if station in weather_files:
            df_station = weather_files[station]

            # Find matching column (handle naming variations)
            matching_cols = [c for c in df_station.columns if col.lower() in c.lower()]

            if matching_cols:
                actual_col = matching_cols[0]

                # Merge on date
                temp_merge = df_station[['date', actual_col]].copy()
                temp_merge[actual_col] = pd.to_numeric(temp_merge[actual_col], errors='coerce')

                # Merge into weighted_weather
                weighted_weather = weighted_weather.merge(
                    temp_merge.rename(columns={actual_col: f'{col}_{station}'}),
                    on='date',
                    how='left'
                )

                # Add weighted contribution
                weighted_weather[f'weighted_{col}'] += (
                    weighted_weather[f'{col}_{station}'] * weight
                )

                # Clean up temporary column
                weighted_weather = weighted_weather.drop(columns=[f'{col}_{station}'])

                print(f"    + {station:15} × {weight:.1%}")

print(f"\n✓ Weighted features created!")
print(f"\nFinal shape: {weighted_weather.shape}")
print(f"Columns: {weighted_weather.columns.tolist()}")


CREATING WEIGHTED WEATHER FEATURES

Base dataframe: 4018 unique dates
Date range: 2014-01-01 to 2024-12-31

Creating weighted features:

  temp_max:

  temp_min:

  temp_mean:

  humidity_max:

  humidity_min:

  precipitation:
    + COAST           × 17.8%
    + EAST            × 2.2%


## Step 5: Verify and Save

In [ ]:
# Check for missing values
print("\n" + "="*80)
print("DATA QUALITY CHECK")
print("="*80)

missing = weighted_weather.isnull().sum()
if missing.sum() > 0:
    print(f"\n⚠️  Missing values found:")
    for col, count in missing[missing > 0].items():
        pct = (count / len(weighted_weather)) * 100
        print(f"  {col}: {count} ({pct:.1f}%)")
else:
    print(f"\n✓ No missing values!")

# Show sample
print(f"\nSample data:")
weighted_cols = [c for c in weighted_weather.columns if 'weighted' in c]
print(weighted_weather[['date'] + weighted_cols[:3]].head(10))

# Statistics
print(f"\nWeighted feature statistics:")
for col in weighted_cols:
    print(f"\n  {col}:")
    print(f"    Mean: {weighted_weather[col].mean():.2f}")
    print(f"    Min:  {weighted_weather[col].min():.2f}")
    print(f"    Max:  {weighted_weather[col].max():.2f}")
    print(f"    Std:  {weighted_weather[col].std():.2f}")

## Step 6: Save Final File

In [ ]:
OUTPUT_FILE = 'final_weighted_weather_features.csv'

print(f"\nSaving to: {OUTPUT_FILE}")

# Keep only date and weighted columns
output_cols = ['date'] + [c for c in weighted_weather.columns if 'weighted' in c]
weighted_weather[output_cols].to_csv(OUTPUT_FILE, index=False)

print(f"✓ Saved!")
print(f"\nFile info:")
print(f"  Rows: {len(weighted_weather):,}")
print(f"  Columns: {len(output_cols)}")
print(f"  Size: {Path(OUTPUT_FILE).stat().st_size / 1024 / 1024:.2f} MB")

print(f"\n" + "="*80)
print(f"✅ PREPROCESSING COMPLETE")
print(f"="*80)
print(f"\nNext steps:")
print(f"1. Upload {OUTPUT_FILE} to your working directory")
print(f"2. In your training notebook, load it with:")
print(f"   weather_df = pd.read_csv('{OUTPUT_FILE}')")
print(f"3. Merge with load data on date")
print(f"4. Train your models!")